In [1]:
import numpy as np
import anndata

from sklearn.decomposition import PCA
from umap import UMAP
from openTSNE import TSNE

/home/jan/anaconda3/envs/elephant_analysis_env2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load datasets

In [2]:
adata_exut=anndata.read_h5ad("../data/preprocessed/exut_adata.h5ad")
adata_sim = anndata.read_h5ad('../data/preprocessed/exut-sim-theta-10-real-seqdepths_adata.h5ad')
adata_merfish=anndata.read_h5ad("../data/preprocessed/merfish_adata.h5ad")
adata_smartseq=anndata.read_h5ad("../data/preprocessed/smartseq_adata.h5ad")
adata_mnist=anndata.read_h5ad("../data/preprocessed/mnist_adata.h5ad")

datasets = [adata_exut,adata_merfish,adata_smartseq,adata_mnist,adata_sim]

In [3]:
#number of classes per dataset
for ad in datasets:
    print(ad.uns['dataset'],len(np.unique(ad.obs['clusterlabels'])),sep=' ')

exut 19
merfish 25
smartseq 28
mnist 10
exut-sim-theta-10-real-seqdepths 19


In [5]:
def compute_embeddings(ad,seeds):
    
    dataset = ad.uns['dataset']
    
    #compute for scaled and unscaled data for main datasets
    if dataset in ['exut','merfish','smartseq']:
        use_scaled_modes = [False, True]
        print('running on scaled and unscaled data for', dataset)
    #only use unscaled for additional datasets
    else:
        use_scaled_modes = [False]
        print('running only on unscaled data for', dataset)
        
    for use_scaled in use_scaled_modes:
        
        if use_scaled:
            x_hd = ad.layers['X_scaled']
            scaled_str = '_scaled'
        else:
            x_hd = ad.X
            scaled_str = ''
        
        for i,seed in enumerate(seeds):

            print(dataset, 'seed:', seed, 'scaled:', use_scaled)

            pca2 = PCA(random_state=seed,n_components=2)
            pca50 = PCA(random_state=seed,n_components=50)
            umap=UMAP(random_state=seed,verbose=True)
            tsne=TSNE(random_state=seed,verbose=True)

            id_str = f'seed_{seed}'

            print('PCA2')
            x_pca2 = pca2.fit_transform(x_hd)
            print('PCA50')
            x_pca50 = pca50.fit_transform(x_hd)
            print('UMAP')
            x_umap = umap.fit_transform(x_pca50)
            print('TSNE')
            x_tsne = tsne.fit(x_pca50)

            ad.obsm[f'x{scaled_str}_pca2_{id_str}'] = x_pca2
            ad.obsm[f'x{scaled_str}_pca50_{id_str}'] = x_pca50
            ad.obsm[f'x{scaled_str}_umap_{id_str}'] = x_umap
            ad.obsm[f'x{scaled_str}_tsne_{id_str}'] = np.array(x_tsne)

            np.save(f'../results/embeddings/npy/{dataset}_x{scaled_str}_pca2_{id_str}.npy',x_pca2,allow_pickle=False)
            np.save(f'../results/embeddings/npy/{dataset}_x{scaled_str}_pca50_{id_str}.npy',x_pca50,allow_pickle=False)
            np.save(f'../results/embeddings/npy/{dataset}_x{scaled_str}_umap_{id_str}.npy',x_umap,allow_pickle=False)
            np.save(f'../results/embeddings/npy/{dataset}_x{scaled_str}_tsne_{id_str}.npy',x_tsne,allow_pickle=False)
    
    ad.uns['seeds']=seeds
    ad.write_h5ad(f'../results/embeddings/{dataset}_adata_standard_embeddings.h5ad')

In [6]:
###NOTE: elephant embeddings are computed in another conda environment (by Chari & Pachter) and separate notebook
[compute_embeddings(ad=ad,seeds=np.arange(5)) for ad in datasets]

running scaled and unscaled for  exut
exut seed: 0 scaled: False
PCA2
PCA50
UMAP
UMAP(random_state=0, verbose=True)
Mon Mar  4 18:13:08 2024 Construct fuzzy simplicial set
Mon Mar  4 18:13:08 2024 Finding Nearest Neighbors
Mon Mar  4 18:13:08 2024 Building RP forest with 9 trees


/home/jan/anaconda3/envs/elephant_analysis_env2/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Mon Mar  4 18:13:10 2024 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	Stopping threshold met -- exiting after 4 iterations
Mon Mar  4 18:13:15 2024 Finished Nearest Neighbor Search
Mon Mar  4 18:13:16 2024 Construct embedding


Epochs completed:   3%| ██████▊                                                                                                                                                                                                                                             14/500 [00:00]

	completed  0  /  500 epochs


...

Epochs completed:  93%| █████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  465/500 [00:04]

	completed  450  /  500 epochs


Epochs completed: 100%| ██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ 500/500 [00:05]


Mon Mar  4 18:13:21 2024 Finished embedding
TSNE
--------------------------------------------------------------------------------
TSNE(early_exaggeration=12, random_state=0, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 1.45 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.09 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.01 seconds
===> Running optimization with exaggeration=12.00, lr=517.08 for 250 iterations...
Iteration   50, KL divergence 3.5645, 50 iterations in 4.7227 sec
Iteration  100, KL divergence 3.4640, 50 iterations in 4.3720 sec


Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7f6db00f2340>
Traceback (most recent call last):
  File "/home/jan/anaconda3/envs/elephant_analysis_env2/lib/python3.11/weakref.py", line 105, in remove
    def remove(wr, selfref=ref(self), _atomic_removal=_remove_dead_weakref):

KeyboardInterrupt: 

KeyboardInterrupt



### Package versions

In [ ]:
np.__version__

In [ ]:
anndata.__version__

In [ ]:
import sklearn; sklearn.__version__

In [ ]:
import umap; umap.__version__

In [ ]:
import openTSNE; openTSNE.__version__